In [48]:
import os
import numpy as np
import pandas as pd
import warnings
warnings.filterwarnings('ignore')
from IPython.display import display
import importlib

In [49]:
activity = pd.read_csv('Activity.csv')
physiology = pd.read_csv('Physiology.csv')
labels = pd.read_csv('Labels.csv')
sleep = pd.read_csv('Sleep.csv')
demographics = pd.read_csv('Demographics.csv')

activity = activity.rename(columns={
    'patient_id': 'id',
    'date': 'date-time',
    'location_name': 'location'})

physiology = physiology.rename(columns={
    'patient_id': 'id',
    'date': 'date-time',
    'device_type': 'type'})

labels = labels.rename(columns={
    'patient_id': 'id',
    'date': 'date-time'})

sleep = sleep.rename(columns={
    'patient_id': 'id',
    'date': 'date-time'})

demographics = demographics.rename(columns={
    'patient_id': 'id'})

# Standardize location names
activity['location'] = activity['location'].replace({
    'Back Door': 'back-door',
    'Bathroom': 'bathroom',
    'Bedroom': 'bedroom',
    'Fridge Door': 'fridge-door',
    'Front Door': 'front-door',
    'Hallway': 'hallway',
    'Kitchen': 'kitchen',
    'Lounge': 'lounge'
})


physiology['type'] = physiology['type'].replace({
    'Body Temperature': 'body-temperature',
    'Systolic blood pressure': 'systolic-blood-pressure',
    'Diastolic blood pressure': 'diastolic-blood-pressure',
    'Heart rate': 'heart-rate',
    'Body weight': 'body-weight',
    'O/E - muscle mass': 'muscle-mass',
    'Total body water': 'total-body-water',
    'Skin Temperature': 'skin-temperature'
})


labels['type'] = labels['type'].replace({
    'Blood pressure': 'blood-pressure',
    'Agitation': 'agitation',
    'Body water': 'body-water',
    'Pulse': 'pulse',
    'Weight': 'weight',
    'Body temperature': 'body-temperature-label'
})


demographics['sex'] = demographics['sex'].map({'Female': 0, 'Male': 1})
demographics['age'] = demographics['age'].map({'(70, 80]': 75, '(80, 90]': 85, '(90, 110]': 100})


In [50]:
print(activity.shape,
      physiology.shape,
      labels.shape,
      sleep.shape,
      demographics.shape)

(1030559, 3) (17680, 5) (608, 3) (461423, 6) (56, 3)


In [51]:
print(activity.columns.values)
print(physiology.columns.values)
print(labels.columns.values)
print(sleep.columns.values)
print(demographics.columns.values)

['id' 'location' 'date-time']
['id' 'date-time' 'type' 'value' 'unit']
['id' 'date-time' 'type']
['id' 'date-time' 'state' 'heart_rate' 'respiratory_rate' 'snoring']
['id' 'age' 'sex']


In [52]:
print(activity.location.unique())
print(physiology.type.unique())
print(labels.type.unique())
print(sleep.state.unique())
print(demographics.age.unique(), demographics.sex.unique())

['fridge-door' 'kitchen' 'front-door' 'bedroom' 'back-door' 'bathroom'
 'lounge' 'hallway']
['body-temperature' 'systolic-blood-pressure' 'diastolic-blood-pressure'
 'heart-rate' 'body-weight' 'muscle-mass' 'total-body-water'
 'skin-temperature']
['blood-pressure' 'agitation' 'body-water' 'pulse' 'weight'
 'body-temperature-label']
['AWAKE' 'LIGHT' 'DEEP' 'REM']
[ 75  85 100] [0 1]


In [53]:
print(activity.id.unique().shape)
print(physiology.id.unique().shape)
print(labels.id.unique().shape)
print(sleep.id.unique().shape)
print(demographics.id.unique().shape)

(56,)
(56,)
(49,)
(17,)
(56,)


In [113]:
# Activity

import utils_data
importlib.reload(utils_data)

from utils_data import extract_6h_count, extract_6h_contextual, extract_6h_statistical

destination = '/home/ali/PycharmProjects/tihm/dataset'

data_activity = []

for idx, (id, group) in enumerate(activity.sort_values('id').groupby('id')):

    group['date-time'] = pd.to_datetime(group['date-time'])
    group = group.sort_values('date-time')
    group['date'] = group['date-time'].dt.date


    daily_count = extract_6h_count(group)
    daily_contextual = extract_6h_contextual(group)
    daily_statistical = extract_6h_statistical(group)

    daily = daily_count.merge(daily_contextual, on=['id', 'date', '6h'], how='outer').merge(daily_statistical, on=['id', 'date', '6h'], how='outer')

    data_activity.append(daily)


data_activity = pd.concat(data_activity, ignore_index=True)
data_activity.to_csv(os.path.join(destination, 'data-06h-activity' + '.csv'), index=False)
print(data_activity.shape)

         id     location           date-time        date
0     0697d  fridge-door 2019-06-28 13:03:29  2019-06-28
1     0697d      kitchen 2019-06-28 13:11:44  2019-06-28
2     0697d   front-door 2019-06-28 13:13:50  2019-06-28
3     0697d      bedroom 2019-06-28 13:13:53  2019-06-28
4     0697d  fridge-door 2019-06-28 13:14:09  2019-06-28
...     ...          ...                 ...         ...
1039  0697d      bedroom 2019-06-30 23:34:32  2019-06-30
1040  0697d      bedroom 2019-06-30 23:36:57  2019-06-30
1041  0697d      bedroom 2019-06-30 23:38:44  2019-06-30
1042  0697d      bedroom 2019-06-30 23:39:49  2019-06-30
1043  0697d      bedroom 2019-06-30 23:44:25  2019-06-30

[1044 rows x 4 columns]
          id     location           date-time        date
1044   099bc      kitchen 2019-05-15 12:19:32  2019-05-15
1045   099bc   front-door 2019-05-15 12:20:20  2019-05-15
1046   099bc  fridge-door 2019-05-15 12:20:21  2019-05-15
1047   099bc    back-door 2019-05-15 12:20:24  2019-05-15
1

In [55]:
print(data_activity.shape, daily.shape, daily_count.shape, daily_contextual.shape, daily_statistical.shape)
# print(daily_count.columns.values)
# print(daily_contextual.columns.values)
# print(daily_statistical.columns.values)
# print(daily.columns.values)
# daily_count.head()
# daily_contextual.head()
# daily_statistical.head()
# daily.head()
display(daily_count)
display(daily_contextual)
display(daily_statistical)
display(daily)
display(data_activity)

(10888, 51) (16, 51) (16, 11) (16, 11) (16, 35)


,id,date,6h,back-door,bathroom,bedroom,fridge-door,front-door,hallway,kitchen,lounge
0,fd100,2019-06-27,00-06,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,fd100,2019-06-27,06-12,2.0,4.0,5.0,21.0,10.0,7.0,24.0,13.0
2,fd100,2019-06-27,12-18,0.0,9.0,17.0,2.0,9.0,20.0,52.0,46.0
3,fd100,2019-06-27,18-24,0.0,11.0,26.0,0.0,6.0,20.0,24.0,38.0
4,fd100,2019-06-28,00-06,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,fd100,2019-06-28,06-12,0.0,22.0,41.0,0.0,16.0,31.0,52.0,45.0
6,fd100,2019-06-28,12-18,0.0,6.0,35.0,7.0,2.0,12.0,69.0,48.0
7,fd100,2019-06-28,18-24,0.0,4.0,15.0,0.0,3.0,15.0,24.0,27.0
8,fd100,2019-06-29,00-06,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9,fd100,2019-06-29,06-12,0.0,10.0,21.0,7.0,3.0,13.0,32.0,16.0


,id,date,6h,total-events,unique-locations,active-location-ratio,private-to-public-ratio,location-entropy,location-dominance-ratio,back-and-forth-count,num-transitions
0,fd100,2019-06-27,00-06,0,0,0.000,1.0000,0.0000,0.0000,0,0
1,fd100,2019-06-27,06-12,86,8,1.000,0.2045,2.6488,0.2791,7,51
2,fd100,2019-06-27,12-18,155,7,0.875,0.2203,2.3375,0.3355,23,120
3,fd100,2019-06-27,18-24,125,6,0.750,0.4512,2.3924,0.3040,14,95
4,fd100,2019-06-28,00-06,0,0,0.000,1.0000,0.0000,0.0000,0,0
5,fd100,2019-06-28,06-12,207,6,0.750,0.4922,2.4814,0.2512,16,164
6,fd100,2019-06-28,12-18,179,7,0.875,0.3178,2.1806,0.3855,12,105
7,fd100,2019-06-28,18-24,88,6,0.750,0.2879,2.2733,0.3068,11,72
8,fd100,2019-06-29,00-06,0,0,0.000,1.0000,0.0000,0.0000,0,0
9,fd100,2019-06-29,06-12,102,7,0.875,0.5082,2.5355,0.3137,7,69


,id,date,6h,back-door-count-max,back-door-count-mean,back-door-count-std,back-door-count-sum,bathroom-count-max,bathroom-count-mean,bathroom-count-std,...,hallway-count-std,hallway-count-sum,kitchen-count-max,kitchen-count-mean,kitchen-count-std,kitchen-count-sum,lounge-count-max,lounge-count-mean,lounge-count-std,lounge-count-sum
0,fd100,2019-06-27,00-06,0,0.0000,0.0000,0,0,0.0000,0.0000,...,0.0000,0,0,0.0000,0.0000,0,0,0.0000,0.0000,0
1,fd100,2019-06-27,06-12,2,0.1176,0.4851,2,3,0.2353,0.7524,...,1.1757,7,21,1.4118,5.0997,24,10,0.7647,2.4882,13
2,fd100,2019-06-27,12-18,0,0.0000,0.0000,0,4,0.5294,1.2307,...,2.6276,20,18,3.0588,5.9316,52,20,2.7059,5.6210,46
3,fd100,2019-06-27,18-24,0,0.0000,0.0000,0,5,0.6471,1.4116,...,2.2146,20,12,1.4118,3.1036,24,11,2.2353,4.0855,38
4,fd100,2019-06-28,00-06,0,0.0000,0.0000,0,0,0.0000,0.0000,...,0.0000,0,0,0.0000,0.0000,0,0,0.0000,0.0000,0
5,fd100,2019-06-28,06-12,0,0.0000,0.0000,0,11,1.2941,3.0774,...,3.5042,31,20,3.0588,6.3391,52,15,2.6471,5.1471,45
6,fd100,2019-06-28,12-18,0,0.0000,0.0000,0,5,0.3529,1.2217,...,2.4178,12,28,4.0588,7.4370,69,13,2.8235,4.3335,48
7,fd100,2019-06-28,18-24,0,0.0000,0.0000,0,3,0.2353,0.7524,...,1.9963,15,13,1.4118,3.6922,24,12,1.5882,3.8089,27
8,fd100,2019-06-29,00-06,0,0.0000,0.0000,0,0,0.0000,0.0000,...,0.0000,0,0,0.0000,0.0000,0,0,0.0000,0.0000,0
9,fd100,2019-06-29,06-12,0,0.0000,0.0000,0,7,0.5882,1.8048,...,2.3326,13,18,1.8824,4.8203,32,8,0.9412,2.1057,16


,id,date,6h,back-door,bathroom,bedroom,fridge-door,front-door,hallway,kitchen,...,hallway-count-std,hallway-count-sum,kitchen-count-max,kitchen-count-mean,kitchen-count-std,kitchen-count-sum,lounge-count-max,lounge-count-mean,lounge-count-std,lounge-count-sum
0,fd100,2019-06-27,00-06,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0000,0,0,0.0000,0.0000,0,0,0.0000,0.0000,0
1,fd100,2019-06-27,06-12,2.0,4.0,5.0,21.0,10.0,7.0,24.0,...,1.1757,7,21,1.4118,5.0997,24,10,0.7647,2.4882,13
2,fd100,2019-06-27,12-18,0.0,9.0,17.0,2.0,9.0,20.0,52.0,...,2.6276,20,18,3.0588,5.9316,52,20,2.7059,5.6210,46
3,fd100,2019-06-27,18-24,0.0,11.0,26.0,0.0,6.0,20.0,24.0,...,2.2146,20,12,1.4118,3.1036,24,11,2.2353,4.0855,38
4,fd100,2019-06-28,00-06,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0000,0,0,0.0000,0.0000,0,0,0.0000,0.0000,0
5,fd100,2019-06-28,06-12,0.0,22.0,41.0,0.0,16.0,31.0,52.0,...,3.5042,31,20,3.0588,6.3391,52,15,2.6471,5.1471,45
6,fd100,2019-06-28,12-18,0.0,6.0,35.0,7.0,2.0,12.0,69.0,...,2.4178,12,28,4.0588,7.4370,69,13,2.8235,4.3335,48
7,fd100,2019-06-28,18-24,0.0,4.0,15.0,0.0,3.0,15.0,24.0,...,1.9963,15,13,1.4118,3.6922,24,12,1.5882,3.8089,27
8,fd100,2019-06-29,00-06,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0000,0,0,0.0000,0.0000,0,0,0.0000,0.0000,0
9,fd100,2019-06-29,06-12,0.0,10.0,21.0,7.0,3.0,13.0,32.0,...,2.3326,13,18,1.8824,4.8203,32,8,0.9412,2.1057,16


,id,date,6h,back-door,bathroom,bedroom,fridge-door,front-door,hallway,kitchen,...,hallway-count-std,hallway-count-sum,kitchen-count-max,kitchen-count-mean,kitchen-count-std,kitchen-count-sum,lounge-count-max,lounge-count-mean,lounge-count-std,lounge-count-sum
0,0697d,2019-06-28,00-06,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0000,0,0,0.0000,0.0000,0,0,0.0000,0.0000,0
1,0697d,2019-06-28,06-12,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0000,0,0,0.0000,0.0000,0,0,0.0000,0.0000,0
2,0697d,2019-06-28,12-18,12.0,5.0,18.0,23.0,27.0,30.0,57.0,...,3.1705,30,20,2.5909,5.4394,57,16,2.2273,4.6999,49
3,0697d,2019-06-28,18-24,2.0,2.0,6.0,0.0,1.0,10.0,49.0,...,1.3355,10,18,2.2273,4.8495,49,16,1.4091,3.8130,31
4,0697d,2019-06-29,00-06,0.0,0.0,3.0,0.0,0.0,0.0,0.0,...,0.0000,0,0,0.0000,0.0000,0,0,0.0000,0.0000,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10883,fd100,2019-06-29,18-24,0.0,13.0,16.0,3.0,3.0,23.0,22.0,...,2.9988,23,10,1.2941,2.7560,22,12,2.0000,3.7583,34
10884,fd100,2019-06-30,00-06,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0000,0,0,0.0000,0.0000,0,0,0.0000,0.0000,0
10885,fd100,2019-06-30,06-12,0.0,13.0,34.0,5.0,8.0,17.0,38.0,...,2.3717,17,13,2.2353,4.4092,38,10,1.8235,3.4683,31
10886,fd100,2019-06-30,12-18,0.0,3.0,29.0,3.0,4.0,6.0,50.0,...,1.0572,6,16,2.9412,5.2852,50,9,1.8824,2.9766,32


In [ ]:
# Physiology

import utils_data
importlib.reload(utils_data)

from utils_data import extract_6h_physiology

data_physiology = []

for idx, (id, group) in enumerate(physiology.sort_values('id').groupby('id')):

    group['date-time'] = pd.to_datetime(group['date-time'])
    group = group.sort_values('date-time')
    group['date'] = group['date-time'].dt.date


    daily_physiology = extract_6h_physiology(group)


    data_physiology.append(daily_physiology)


data_physiology = pd.concat(data_physiology, ignore_index=True)
data_physiology.to_csv(os.path.join(destination, 'data-06h-physiology' + '.csv'), index=False)

,id,date-time,type,value,unit,date
0,0697d,2019-06-28 13:42:09,body-temperature,36.072,Cel,2019-06-28
1,0697d,2019-06-28 13:42:44,body-temperature,35.831,Cel,2019-06-28
2,0697d,2019-06-28 13:43:40,body-temperature,35.831,Cel,2019-06-28
4,0697d,2019-06-28 13:45:15,diastolic-blood-pressure,82.000,mm[Hg],2019-06-28
5,0697d,2019-06-28 13:45:15,heart-rate,42.000,beats/min,2019-06-28
3,0697d,2019-06-28 13:45:15,systolic-blood-pressure,165.000,mm[Hg],2019-06-28
6,0697d,2019-06-28 14:25:07,body-temperature,36.599,Cel,2019-06-28
7,0697d,2019-06-28 14:25:37,body-temperature,36.599,Cel,2019-06-28
10,0697d,2019-06-28 14:40:13,total-body-water,50.600,%,2019-06-28
8,0697d,2019-06-28 14:40:13,body-weight,86.300,kg,2019-06-28


,id,date-time,type,value,unit,date
37,099bc,2019-05-15 11:53:13,body-weight,53.300,kg,2019-05-15
38,099bc,2019-05-15 11:53:13,total-body-water,54.000,%,2019-05-15
39,099bc,2019-05-15 11:53:13,muscle-mass,39.800,kg,2019-05-15
40,099bc,2019-05-15 12:11:16,body-temperature,36.499,Cel,2019-05-15
41,099bc,2019-05-15 12:11:35,body-temperature,36.464,Cel,2019-05-15
...,...,...,...,...,...,...
490,099bc,2019-06-30 08:36:27,body-temperature,37.314,Cel,2019-06-30
491,099bc,2019-06-30 08:36:48,body-temperature,37.314,Cel,2019-06-30
493,099bc,2019-06-30 15:39:36,diastolic-blood-pressure,76.000,mm[Hg],2019-06-30
492,099bc,2019-06-30 15:39:36,systolic-blood-pressure,123.000,mm[Hg],2019-06-30


,id,date-time,type,value,unit,date
495,0cda9,2019-04-15 13:22:50,heart-rate,75.0,beats/min,2019-04-15


,id,date-time,type,value,unit,date
496,0d5ef,2019-05-13 10:13:45,body-temperature,36.759,Cel,2019-05-13
497,0d5ef,2019-05-13 10:14:43,body-temperature,36.338,Cel,2019-05-13
498,0d5ef,2019-05-13 10:15:21,body-temperature,36.338,Cel,2019-05-13
499,0d5ef,2019-05-13 10:15:31,body-temperature,36.226,Cel,2019-05-13
500,0d5ef,2019-05-13 10:15:56,body-temperature,36.226,Cel,2019-05-13
...,...,...,...,...,...,...
773,0d5ef,2019-06-30 15:37:38,body-temperature,36.343,Cel,2019-06-30
774,0d5ef,2019-06-30 15:37:58,body-temperature,36.343,Cel,2019-06-30
775,0d5ef,2019-06-30 15:39:07,systolic-blood-pressure,139.000,mm[Hg],2019-06-30
776,0d5ef,2019-06-30 15:39:07,diastolic-blood-pressure,78.000,mm[Hg],2019-06-30


,id,date-time,type,value,unit,date
778,0efe8,2019-05-14 14:38:48,body-temperature,36.978,Cel,2019-05-14
779,0efe8,2019-05-14 14:39:19,body-temperature,36.565,Cel,2019-05-14
780,0efe8,2019-05-14 14:40:06,body-temperature,36.565,Cel,2019-05-14
781,0efe8,2019-05-14 14:42:36,systolic-blood-pressure,123.000,mm[Hg],2019-05-14
782,0efe8,2019-05-14 14:42:36,diastolic-blood-pressure,103.000,mm[Hg],2019-05-14
...,...,...,...,...,...,...
1150,0efe8,2019-06-30 08:39:54,systolic-blood-pressure,143.000,mm[Hg],2019-06-30
1151,0efe8,2019-06-30 08:39:54,diastolic-blood-pressure,71.000,mm[Hg],2019-06-30
1152,0efe8,2019-06-30 08:39:54,heart-rate,64.000,beats/min,2019-06-30
1153,0efe8,2019-06-30 08:40:50,body-temperature,36.987,Cel,2019-06-30


...

,id,date-time,type,value,unit,date
16171,e8a78,2019-04-16 12:02:32,diastolic-blood-pressure,94.000,mm[Hg],2019-04-16
16170,e8a78,2019-04-16 12:02:32,systolic-blood-pressure,149.000,mm[Hg],2019-04-16
16172,e8a78,2019-04-16 12:02:32,heart-rate,79.000,beats/min,2019-04-16
16173,e8a78,2019-04-16 12:03:36,body-temperature,36.192,Cel,2019-04-16
16174,e8a78,2019-04-16 12:04:16,body-temperature,36.192,Cel,2019-04-16
...,...,...,...,...,...,...
16459,e8a78,2019-06-30 08:24:09,heart-rate,62.000,beats/min,2019-06-30
16457,e8a78,2019-06-30 08:24:09,systolic-blood-pressure,114.000,mm[Hg],2019-06-30
16458,e8a78,2019-06-30 08:24:09,diastolic-blood-pressure,64.000,mm[Hg],2019-06-30
16460,e8a78,2019-06-30 14:43:52,body-temperature,37.323,Cel,2019-06-30


,id,date-time,type,value,unit,date
16462,ec812,2019-04-02 13:53:16,body-temperature,37.398,Cel,2019-04-02
16463,ec812,2019-04-02 13:53:47,body-temperature,37.398,Cel,2019-04-02
16464,ec812,2019-04-08 16:42:19,body-temperature,35.434,Cel,2019-04-08
16465,ec812,2019-04-08 16:42:38,body-temperature,36.705,Cel,2019-04-08
16466,ec812,2019-04-08 16:46:37,body-temperature,36.439,Cel,2019-04-08
...,...,...,...,...,...,...
17098,ec812,2019-06-25 09:10:19,body-temperature,36.511,Cel,2019-06-25
17099,ec812,2019-06-25 09:10:58,body-temperature,36.511,Cel,2019-06-25
17100,ec812,2019-06-26 08:22:10,body-temperature,36.004,Cel,2019-06-26
17101,ec812,2019-06-27 15:38:02,body-temperature,36.288,Cel,2019-06-27


,id,date-time,type,value,unit,date
17103,eca1f,2019-05-07 15:05:27,systolic-blood-pressure,119.000,mm[Hg],2019-05-07
17105,eca1f,2019-05-07 15:05:27,heart-rate,99.000,beats/min,2019-05-07
17104,eca1f,2019-05-07 15:05:27,diastolic-blood-pressure,77.000,mm[Hg],2019-05-07
17106,eca1f,2019-05-07 15:06:28,body-temperature,36.642,Cel,2019-05-07
17107,eca1f,2019-05-07 15:06:56,body-temperature,36.615,Cel,2019-05-07
...,...,...,...,...,...,...
17472,eca1f,2019-06-30 14:33:04,diastolic-blood-pressure,71.000,mm[Hg],2019-06-30
17473,eca1f,2019-06-30 14:33:04,heart-rate,87.000,beats/min,2019-06-30
17474,eca1f,2019-06-30 14:36:14,muscle-mass,48.300,kg,2019-06-30
17475,eca1f,2019-06-30 14:36:14,body-weight,65.700,kg,2019-06-30


,id,date-time,type,value,unit,date
17478,f220c,2019-04-26 10:25:59,skin-temperature,34.435,Cel,2019-04-26
17477,f220c,2019-04-26 10:25:59,body-temperature,36.701,Cel,2019-04-26
17480,f220c,2019-04-26 11:03:02,skin-temperature,33.988,Cel,2019-04-26
17479,f220c,2019-04-26 11:03:02,body-temperature,36.312,Cel,2019-04-26
17481,f220c,2019-04-26 11:25:59,body-temperature,36.701,Cel,2019-04-26
...,...,...,...,...,...,...
17648,f220c,2019-06-29 11:32:02,body-temperature,37.027,Cel,2019-06-29
17649,f220c,2019-06-29 11:32:41,body-temperature,37.027,Cel,2019-06-29
17650,f220c,2019-06-29 11:35:46,systolic-blood-pressure,110.000,mm[Hg],2019-06-29
17652,f220c,2019-06-29 11:35:46,heart-rate,84.000,beats/min,2019-06-29


,id,date-time,type,value,unit,date
17653,fd100,2019-06-27 11:11:02,body-temperature,36.686,Cel,2019-06-27
17654,fd100,2019-06-27 11:30:26,systolic-blood-pressure,144.000,mm[Hg],2019-06-27
17656,fd100,2019-06-27 11:30:26,heart-rate,55.000,beats/min,2019-06-27
17655,fd100,2019-06-27 11:30:26,diastolic-blood-pressure,84.000,mm[Hg],2019-06-27
17657,fd100,2019-06-27 11:32:21,systolic-blood-pressure,162.000,mm[Hg],2019-06-27
17658,fd100,2019-06-27 11:32:21,diastolic-blood-pressure,68.000,mm[Hg],2019-06-27
17659,fd100,2019-06-27 11:32:21,heart-rate,53.000,beats/min,2019-06-27
17660,fd100,2019-06-27 11:33:24,body-temperature,36.903,Cel,2019-06-27
17661,fd100,2019-06-27 11:33:59,body-temperature,36.503,Cel,2019-06-27
17662,fd100,2019-06-27 11:34:23,body-temperature,36.903,Cel,2019-06-27


In [57]:
print(data_physiology.shape, daily_physiology.shape)
print(daily_physiology.columns.values)
display(daily_physiology)
display(data_physiology)

(2578, 11) (3, 11)
['id' 'date' '6h' 'body-temperature' 'body-weight'
 'diastolic-blood-pressure' 'heart-rate' 'muscle-mass'
 'systolic-blood-pressure' 'total-body-water' 'skin-temperature']


,id,date,6h,body-temperature,body-weight,diastolic-blood-pressure,heart-rate,muscle-mass,systolic-blood-pressure,total-body-water,skin-temperature
0,fd100,2019-06-27,06-12,36.6996,55.8,76.0,54.0,37.5,153.0,48.7,<NA>
1,fd100,2019-06-28,18-24,36.6784,NaN,74.0,61.0,NaN,138.0,NaN,<NA>
2,fd100,2019-06-29,18-24,36.2400,NaN,NaN,NaN,NaN,NaN,NaN,<NA>


,id,date,6h,body-temperature,body-weight,diastolic-blood-pressure,heart-rate,muscle-mass,systolic-blood-pressure,total-body-water,skin-temperature
0,0697d,2019-06-28,12-18,36.1864,86.3,82.0,42.0,64.5,165.0,50.6,NaN
1,0697d,2019-06-29,12-18,36.9148,86.1,78.5,60.0,64.6,157.0,51.2,NaN
2,0697d,2019-06-30,12-18,35.9532,NaN,80.5,52.5,NaN,152.0,NaN,NaN
3,099bc,2019-05-15,06-12,NaN,53.3,NaN,NaN,39.8,NaN,54.0,NaN
4,099bc,2019-05-15,12-18,37.2293,NaN,75.5,82.5,NaN,127.0,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...
2573,f220c,2019-06-23,12-18,36.2770,NaN,70.0,73.0,NaN,111.0,NaN,33.730
2574,f220c,2019-06-29,06-12,37.0362,NaN,68.0,84.0,NaN,110.0,NaN,35.381
2575,fd100,2019-06-27,06-12,36.6996,55.8,76.0,54.0,37.5,153.0,48.7,NaN
2576,fd100,2019-06-28,18-24,36.6784,NaN,74.0,61.0,NaN,138.0,NaN,NaN


In [58]:
# Label

import utils_data
importlib.reload(utils_data)

from utils_data import extract_6h_labels

data_labels = []

for idx, (id, group) in enumerate(labels.sort_values('id').groupby('id')):

    group['date-time'] = pd.to_datetime(group['date-time'])
    group = group.sort_values('date-time')
    group['date'] = group['date-time'].dt.date


    daily_labels = extract_6h_labels(group)

    data_labels.append(daily_labels)


data_labels = pd.concat(data_labels, ignore_index=True)
data_labels.to_csv(os.path.join(destination, 'data-06h-labels' + '.csv'), index=False)

In [59]:
print(data_labels['agitation'].sum())
print(data_labels.shape, daily_labels.shape)
print(daily_labels.columns.values)
display(daily_labels)
display(data_labels)

135.0
(1900, 9) (4, 9)
['id' 'date' '6h' 'blood-pressure' 'agitation' 'body-water' 'pulse'
 'weight' 'body-temperature-label']


,id,date,6h,blood-pressure,agitation,body-water,pulse,weight,body-temperature-label
0,fd100,2019-06-27,00-06,0.0,0,0,0,0,0
1,fd100,2019-06-27,06-12,1.0,0,0,0,0,0
2,fd100,2019-06-27,12-18,0.0,0,0,0,0,0
3,fd100,2019-06-27,18-24,0.0,0,0,0,0,0


,id,date,6h,blood-pressure,agitation,body-water,pulse,weight,body-temperature-label
0,0697d,2019-06-28,00-06,0.0,0.0,0.0,0.0,0.0,0.0
1,0697d,2019-06-28,06-12,0.0,0.0,0.0,0.0,0.0,0.0
2,0697d,2019-06-28,12-18,1.0,0.0,0.0,1.0,0.0,0.0
3,0697d,2019-06-28,18-24,0.0,0.0,0.0,0.0,0.0,0.0
4,0697d,2019-06-29,00-06,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...
1895,eca1f,2019-06-19,18-24,0.0,0.0,0.0,0.0,0.0,0.0
1896,fd100,2019-06-27,00-06,0.0,0.0,0.0,0.0,0.0,0.0
1897,fd100,2019-06-27,06-12,1.0,0.0,0.0,0.0,0.0,0.0
1898,fd100,2019-06-27,12-18,0.0,0.0,0.0,0.0,0.0,0.0


In [60]:
import utils_data
importlib.reload(utils_data)

from utils_data import add_agitation_next_6h

data = data_activity.merge(
    data_physiology, on=['id', 'date', '6h'], how='outer').merge(
        data_labels, on=['id', 'date', '6h'], how='outer').merge(
            demographics, on='id', how='left')

data = add_agitation_next_6h(data)
data = data[data['agitation-next'] != -10]


data.to_csv(os.path.join(destination, 'data-06h' + '.csv'), index=False)
display(data)
print(data.columns)
print(data['agitation'].sum(), data['agitation-next'].sum())

,id,date,6h,back-door,bathroom,bedroom,fridge-door,front-door,hallway,kitchen,...,blood-pressure,agitation,body-water,pulse,weight,body-temperature-label,age,sex,agitation-next,agitation-four
0,0697d,2019-06-28,00-06,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,85,1,0.0,0
1,0697d,2019-06-28,06-12,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,85,1,0.0,0
2,0697d,2019-06-28,12-18,12.0,5.0,18.0,23.0,27.0,30.0,57.0,...,1.0,0.0,0.0,1.0,0.0,0.0,85,1,0.0,0
3,0697d,2019-06-28,18-24,2.0,2.0,6.0,0.0,1.0,10.0,49.0,...,0.0,0.0,0.0,0.0,0.0,0.0,85,1,0.0,0
4,0697d,2019-06-29,00-06,0.0,0.0,3.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,85,1,0.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10979,fd100,2019-06-29,12-18,0.0,10.0,19.0,17.0,9.0,25.0,56.0,...,NaN,NaN,NaN,NaN,NaN,NaN,100,0,NaN,-10
10980,fd100,2019-06-29,18-24,0.0,13.0,16.0,3.0,3.0,23.0,22.0,...,NaN,NaN,NaN,NaN,NaN,NaN,100,0,NaN,-10
10981,fd100,2019-06-30,00-06,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,100,0,NaN,-10
10982,fd100,2019-06-30,06-12,0.0,13.0,34.0,5.0,8.0,17.0,38.0,...,NaN,NaN,NaN,NaN,NaN,NaN,100,0,NaN,-10


Index(['id', 'date', '6h', 'back-door', 'bathroom', 'bedroom', 'fridge-door',
       'front-door', 'hallway', 'kitchen', 'lounge', 'total-events',
       'unique-locations', 'active-location-ratio', 'private-to-public-ratio',
       'location-entropy', 'location-dominance-ratio', 'back-and-forth-count',
       'num-transitions', 'back-door-count-max', 'back-door-count-mean',
       'back-door-count-std', 'back-door-count-sum', 'bathroom-count-max',
       'bathroom-count-mean', 'bathroom-count-std', 'bathroom-count-sum',
       'bedroom-count-max', 'bedroom-count-mean', 'bedroom-count-std',
       'bedroom-count-sum', 'fridge-door-count-max', 'fridge-door-count-mean',
       'fridge-door-count-std', 'fridge-door-count-sum',
       'front-door-count-max', 'front-door-count-mean', 'front-door-count-std',
       'front-door-count-sum', 'hallway-count-max', 'hallway-count-mean',
       'hallway-count-std', 'hallway-count-sum', 'kitchen-count-max',
       'kitchen-count-mean', 'kitchen-coun

In [ ]:
# _data_ = sleep
_data_ = physiology
# _data_ = activity

print(_data_.columns)
# display(_data_)
print(_data_.isna().sum().sum())


_data_['date-time'] = pd.to_datetime(_data_['date-time'])
_data_ = _data_.sort_values('date-time')
_data_['date'] = _data_['date-time'].dt.date

display(_data_['id'].nunique())
display(_data_['date'].nunique())
display(np.round((2803-_data_[['id', 'date']].drop_duplicates().shape[0]) / 2803 * 100, 2))




# types = ['body-temperature', 'systolic-blood-pressure', 'diastolic-blood-pressure', 'heart-rate',
#          'body-weight', 'muscle-mass', 'total-body-water', 'skin-temperature']
# for i, group in _data_.groupby(['id', 'date']):
#     print(
#         group.shape,
#         group[group['type'] == types[0]].shape
#         )
#     display(group[group['type'] == types[0]])


Index(['id', 'date-time', 'type', 'value', 'unit'], dtype='object')
0
